# Constrained single objective optimisation

- run 7010, 2098 separately
- run reparately for cost, mass

### combined: single objective cost / mass

- exchange rate r = 300 €/kg
- minimise m + c/r or c + m*r ???

In [3]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.optimize import SR1

from beam import Cantilever
from functools import partial

## funktion

In [44]:
L = 1000
dstab = None
loads = [16000, -30000]
matname = 'AL7010'
exr = -300 # €/kg, exchange rate

# x: [h, tw, blf, tlf, buf, tuf]
bounds = Bounds([10, 1.5, 6, 0, 6, 0], [300, 12, 100, 12, 100, 12])
x0 = [180, 3.2, 50, 6, 50, 6]

rfkeys = [
    'rf_t_uf', 
    'rf_t_lf',
    'rf_c_uf',
    'rf_c_lf',
    'rf_lb_uf',
    'rf_lb_lf',
    'rf_s_web',
    'rf_wb',
    'rf_lat',
]


def mass(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    return b.mass()

def cost(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    return b.cost()

def combined(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    return b.cost() - exr*b.mass()

# nonlinear constraint functions
def cons_f(x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    result = b.analyse(loads)
    return [min(result[key], 9999) for key in rfkeys]

def rf_constraint(fmode, x):
    b = Cantilever(matname, L, *x, dstab=dstab)
    func = getattr(b, fmode)
    results = [9999] + [func(F) for F in loads]
    return min(results)


# TODO: formulate linear constraint
# bf >= tw
#linear_constraint = LinearConstraint([[1, 2], [2, 1]], [-np.inf, 1], [1, 1])

# all rf >= 1
nonlinear_constraints = [
    #NonlinearConstraint(cons_f, 1, np.inf),
    NonlinearConstraint(partial(rf_constraint, key), 1, np.inf)
    for key in rfkeys
]

### optimise mass

In [45]:
res_m = minimize(mass, x0, method='trust-constr',  jac="2-point", hess=SR1(),
               constraints=nonlinear_constraints,
               options={'verbose': 1}, bounds=bounds)



print(res_m)

`gtol` termination condition is satisfied.
Number of iterations: 150, function evaluations: 1799, CG iterations: 191, optimality: 4.19e-09, constraint violation: 0.00e+00, execution time:  3.9 s.
 barrier_parameter: 1.0240000000000006e-08
 barrier_tolerance: 1.0240000000000006e-08
          cg_niter: 191
      cg_stop_cond: 4
            constr: [array([1.00000001]), array([1.90881421]), array([1.66270627]), array([1.00000002]), array([1.00000397]), array([68.17630958]), array([3.46535325]), array([1.00000003]), array([1.83487141]), array([185.96081158,   3.09978118,  23.96073905,  11.99999898,
        69.88464206,   3.42838559])]
       constr_nfev: [1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 1799, 0]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 3.8653342723846436
               fun: 2.977168616975813
              grad: array([0.00874138, 0.48090143, 0.033

In [46]:
bm = Cantilever('AL7010', L, *res_m.x)
print(bm.mass(), bm.cost())
bm.analyse(loads)

2.977168616975813 398.36481743348713


{'wmax': 25.773436813800018,
 'rf_t_uf': 1.0000000133295703,
 'rf_t_lf': 1.9088142091495075,
 'rf_c_uf': 1.6627062661755485,
 'rf_c_lf': 1.0000000151419373,
 'rf_lb_uf': 1.0000039692616527,
 'rf_lb_lf': 68.17630958164568,
 'rf_s_web': 3.465353253281223,
 'rf_wb': 1.0000000257567954,
 'rf_lat': 1.8348714109334032,
 'Fmax': 30000,
 'mass': 2.977168616975813,
 'cost': 398.36481743348713,
 'area': 1055.7335521190826,
 'L': 1000,
 'h': 185.96081158167237,
 'tw': 3.0997811817767125,
 'blf': 23.96073904928083,
 'tlf': 11.999998982212725,
 'buf': 69.88464206261604,
 'tuf': 3.428385589674079,
 'matname': 'AL7010'}

### optimise cost

In [47]:
res_c = minimize(cost, x0, method='trust-constr',  jac="2-point", #hess=SR1(),
               constraints=nonlinear_constraints,
               options={'verbose': 1}, bounds=bounds)

print(res_c)

c:\program files\python38\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`xtol` termination condition is satisfied.
Number of iterations: 647, function evaluations: 8561, CG iterations: 772, optimality: 2.00e+00, constraint violation: 0.00e+00, execution time: 1.6e+01 s.
 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 772
      cg_stop_cond: 2
            constr: [array([1.04028625]), array([1.95110816]), array([1.92607828]), array([1.02977948]), array([9998.99996853]), array([5480.94704773]), array([11.44035721]), array([23.74092814]), array([4.58340506]), array([168.79260846,  11.96252522,  14.2230313 ,  11.53200596,
        14.22303129,  11.37675245])]
       constr_nfev: [8561, 8561, 8561, 8561, 8561, 8561, 8561, 8561, 8561, 0]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 15.94380235671997
               fun: 71.07978631358847
              grad: array([ 0.40503961, -2.056962

In [48]:
bc = Cantilever('AL7010', L, *res_c.x)
print(bc.mass(), bc.cost())
bc.analyse(loads)

5.840138847834993 71.07978631358847


{'wmax': 27.535168314671054,
 'rf_t_uf': 1.0402862489464597,
 'rf_t_lf': 1.951108162165827,
 'rf_c_uf': 1.9260782840828685,
 'rf_c_lf': 1.029779476340058,
 'rf_lb_uf': 9998.999968529002,
 'rf_lb_lf': 5480.9470477276545,
 'rf_s_web': 11.440357206107738,
 'rf_wb': 23.74092814007761,
 'rf_lat': 4.583405063152204,
 'Fmax': 30000,
 'mass': 5.840138847834993,
 'cost': 71.07978631358847,
 'area': 2070.9712226365223,
 'L': 1000,
 'h': 168.79260845535828,
 'tw': 11.962525216864327,
 'blf': 14.223031297271964,
 'tlf': 11.532005962075289,
 'buf': 14.223031286302902,
 'tuf': 11.37675245384809,
 'matname': 'AL7010'}

## Optimise combined

In [50]:
res_cm = minimize(combined, x0, method='trust-constr',  jac="2-point", hess=SR1(),
               constraints=nonlinear_constraints,
               options={'verbose': 1}, bounds=bounds)

print(res_cm)

`xtol` termination condition is satisfied.
Number of iterations: 603, function evaluations: 7189, CG iterations: 791, optimality: 2.40e+00, constraint violation: 0.00e+00, execution time: 1.5e+01 s.
 barrier_parameter: 2.048000000000001e-09
 barrier_tolerance: 2.048000000000001e-09
          cg_niter: 791
      cg_stop_cond: 4
            constr: [array([1.00010157]), array([1.92916617]), array([1.79691869]), array([1.00006774]), array([140.8503262]), array([90.96695816]), array([3.74687363]), array([1.00018897]), array([2.30916755]), array([201.17767771,   3.20886573,  21.36372164,  11.99946471,
        21.36372161,  11.06013569])]
       constr_nfev: [7189, 7189, 7189, 7189, 7189, 7189, 7189, 7189, 7189, 0]
       constr_nhev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
       constr_njev: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    constr_penalty: 1.0
  constr_violation: 0.0
    execution_time: 14.785714387893677
               fun: 1046.13386594507
              grad: array([  3.44574224, 148.17642617, 

In [52]:
bcm = Cantilever('AL7010', L, *res_cm.x)
print(bcm.mass(), bcm.cost())
bcm.analyse(loads)

3.001032374255838 145.82415366831867


{'wmax': 23.69819713078809,
 'rf_t_uf': 1.0001015746673427,
 'rf_t_lf': 1.9291661686116273,
 'rf_c_uf': 1.7969186937289294,
 'rf_c_lf': 1.000067740555923,
 'rf_lb_uf': 140.85032620007897,
 'rf_lb_lf': 90.96695816230493,
 'rf_s_web': 3.746873625582107,
 'rf_wb': 1.0001889700409239,
 'rf_lat': 2.309167550578753,
 'Fmax': 30000,
 'mass': 3.001032374255838,
 'cost': 145.82415366831867,
 'area': 1064.1958773956871,
 'L': 1000,
 'h': 201.17767771190483,
 'tw': 3.208865727396388,
 'blf': 21.36372164464666,
 'tlf': 11.999464705979447,
 'buf': 21.363721607875203,
 'tuf': 11.060135688737672,
 'matname': 'AL7010'}

In [58]:
45* 60* 1.4/8

472.49999999999994